In [1]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import itertools
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# Текстовая модальность

In [3]:
text_data = pd.read_csv("/content/feature_data.csv")
text_data.head()

,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Column1,data_id,...,audio_paths,dominance_auto,emotions,dominance_1,dominance_2,dominance_3,cleaned_tokens,lemmas,word2vec,fasttext
0,0,0,7211,7211,7211,7211,7211,7211,7211,в2_гр2,...,dataset/2 Зоопарк/audio/в2_гр2/SPEAKER_02_0.09...,low,Апатия,neut,neut,neut,"['понимаю', 'нам', 'нужно', 'подобрать', 'аргу...","['понимать', 'мы', 'нужно', 'подобрать', 'аргу...",[-0.10532451 0.09293796 -0.00918578 -0.020776...,[-0.1132676 -0.03302962 0.02209663 0.065832...
1,1,1,7212,7212,7212,7212,7212,7212,7212,в2_гр2,...,dataset/2 Зоопарк/audio/в2_гр2/SPEAKER_01_4.09...,neut,Нейтральное состояние,neut,low,low,"['почему', 'зоопарк']","['почему', 'зоопарк']",[-0.09030008 0.06780697 -0.00628105 -0.012919...,[-0.11342623 -0.03297137 0.01974361 0.066424...
2,2,2,7213,7213,7213,7213,7213,7213,7213,в2_гр2,...,dataset/2 Зоопарк/audio/в2_гр2/SPEAKER_01_6.31...,high,Спокойная уверенность,neut,neut,neut,"['почему', 'зоопарк', 'это', 'плохо']","['почему', 'зоопарк', 'это', 'плохо']",[-0.10819359 0.08673862 -0.00731089 -0.012298...,[-0.11288241 -0.03260336 0.02027969 0.067807...
3,3,3,7214,7214,7214,7214,7214,7214,7214,в2_гр2,...,dataset/2 Зоопарк/audio/в2_гр2/SPEAKER_01_7.80...,neut,Нейтральное состояние,low,neut,low,"['желательно', 'источниками', 'подобрать', 'ск...","['желательно', 'источник', 'подобрать', 'скаже...",[-0.07795323 0.07166173 -0.00911281 -0.010702...,[-1.13055199e-01 -3.26835997e-02 2.12363023e-...
4,4,4,7215,7215,7215,7215,7215,7215,7215,в2_гр2,...,dataset/2 Зоопарк/audio/в2_гр2/SPEAKER_02_29.6...,neut,Нейтральное состояние,neut,neut,neut,"['целом', 'общих', 'пунктов', 'это', 'любом', ...","['целое', 'общий', 'пункт', 'это', 'любой', 'с...",[-0.11720365 0.10082722 -0.01121727 -0.019163...,[-0.11345374 -0.03313726 0.02094814 0.067507...


### Обучение нейронной сети

Подготовка данных

In [4]:
# Преобразуем строковые векторы в массивы
text_data['word2vec'] = text_data['word2vec'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))
text_data['fasttext'] = text_data['fasttext'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))

# Объединяем признаки
text_data['features'] = text_data.apply(lambda row: np.concatenate([row['word2vec'], row['fasttext']]), axis=1)

# Переводим метки в числовой формат
bin_map = {'neut': 0, 'nonneut': 1}
text_data['fin_binery'] = text_data['fin_binery'].map(bin_map)

Обработка обучающей и тестовой выборку

In [5]:
# Разделяем признаки и метки
X = np.stack(text_data['features'].values)
y = text_data['fin_binery'].values

# Делим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=56)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование в тензоры PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Загрузка в DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)

Определение наборов гиперпараметров

In [6]:
hidden_sizes = [(128, 64, 32), (64, 32, 16)]
learning_rates = [0.001, 0.0005]
batch_sizes = [8, 16]

# Все комбинации
param_grid = list(itertools.product(hidden_sizes, learning_rates, batch_sizes))

best_f1 = 0
best_model = None
best_history = None
best_params = None

Определение архитектуры и обучение модели

In [7]:
def train_text_model(hidden_layers, lr, batch_size, epochs=50):
    class Net(nn.Module):
        def __init__(self, input_dim):
            super(Net, self).__init__()
            self.layers = nn.Sequential(
                nn.Linear(input_dim, hidden_layers[0]),
                nn.BatchNorm1d(hidden_layers[0]),  # Добавил BatchNorm для ускорения обучения
                nn.ReLU(),
                nn.Linear(hidden_layers[0], hidden_layers[1]),
                nn.BatchNorm1d(hidden_layers[1]),  # BatchNorm между слоями
                nn.Dropout(0.3),
                nn.ReLU(),
                nn.Linear(hidden_layers[1], hidden_layers[2]),
                nn.BatchNorm1d(hidden_layers[2]),  # BatchNorm перед последним слоем
                nn.ReLU(),
                nn.Linear(hidden_layers[2], 3)
            )

        def forward(self, x):
            return self.layers(x)

    model = Net(X.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)  # Добавил L2 регуляризацию
    criterion = nn.CrossEntropyLoss()

    # Добавим шедулер для динамического изменения learning rate
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    history = {'loss': [], 'acc': []}

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        correct = 0
        total = 0

        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)

        # Обновляем learning rate на основе потерь
        scheduler.step(epoch_loss / len(train_loader))

        epoch_loss = epoch_loss / len(train_loader)
        epoch_acc = correct / total
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)

    return model, history

In [8]:
# Поиск лучшей модели
best_f1 = 0
best_model = None
best_history = None
best_params = {}

param_grid = [
    # Разные архитектуры + learning rates + batch sizes
    ([512, 256, 128, 64], 0.001, 32),    # Глубокая сеть
    ([256, 128, 64, 32], 0.0005, 64),    # Средняя глубина
    ([128, 64, 32, 16], 0.0001, 128),    # Меньше нейронов, мелкая сеть
    ([256, 128, 64], 0.001, 32),         # Базовый вариант
    ([512, 256, 128], 0.0005, 64),       # Больше нейронов
    ([64, 32, 16], 0.0001, 128),         # Компактная модель
]

for hidden, lr, batch in param_grid:
    print(f"\nТренировка модели с параметрами: hidden_layers={hidden}, lr={lr}, batch_size={batch}")
    model, history = train_text_model(hidden, lr, batch)

    # Оценка модели
    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor)
        pred_labels = torch.argmax(preds, dim=1).numpy()
        f1 = f1_score(y_test, pred_labels, average='macro')
        accuracy = accuracy_score(y_test, pred_labels)

        # Вывод метрик для текущей модели
        print(f"Результаты:")
        print(f"F1-score (macro): {f1:.4f}")
        print(f"Accuracy: {accuracy:.4f}")
        print(classification_report(y_test, pred_labels, target_names=['0', '1']))

    if f1 > best_f1:
        best_f1 = f1
        best_model = model
        best_history = history
        best_params = {'hidden_layers': hidden, 'lr': lr, 'batch_size': batch}
        print("!!! Новая лучшая модель !!!")

# Визуализация обучения лучшей модели
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(best_history['loss'], label='Train Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(best_history['acc'], label='Train Accuracy', color='green')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

# Вывод результатов лучшей модели
print("\n========================================")
print("Лучшая модель по F1-score (macro):")
print(f"F1-score: {best_f1:.4f}")
print(f"Параметры: {best_params}")


Тренировка модели с параметрами: hidden_layers=[512, 256, 128, 64], lr=0.001, batch_size=32
Результаты:
F1-score (macro): 0.4540
Accuracy: 0.4545
              precision    recall  f1-score   support

           0       0.47      0.41      0.44        51
           1       0.44      0.50      0.47        48

    accuracy                           0.45        99
   macro avg       0.46      0.46      0.45        99
weighted avg       0.46      0.45      0.45        99

!!! Новая лучшая модель !!!

Тренировка модели с параметрами: hidden_layers=[256, 128, 64, 32], lr=0.0005, batch_size=64
Результаты:
F1-score (macro): 0.4532
Accuracy: 0.4545
              precision    recall  f1-score   support

           0       0.47      0.49      0.48        51
           1       0.43      0.42      0.43        48

    accuracy                           0.45        99
   macro avg       0.45      0.45      0.45        99
weighted avg       0.45      0.45      0.45        99


Тренировка модели с пар

ValueError: Number of classes, 3, does not match size of target_names, 2. Try specifying the labels parameter

In [ ]:
# Метрики лучшей модели
best_model.eval()
with torch.no_grad():
    final_preds = best_model(X_test_tensor)
    final_labels = torch.argmax(final_preds, dim=1).numpy()

print("\nПолный отчет по метрикам:\n")
print(classification_report(y_test, final_labels, target_names=['0', '1']))

# Аудио модальность

In [ ]:
audio_data = pd.read_csv("/content/audio_data.csv")
audio_data.head()

### Обучение нейронной сети

Подготовка данных

In [ ]:
# Преобразуем строковые векторы в массивы
audio_data['mfcc_mean'] = audio_data['mfcc_mean'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))
audio_data['mfcc_std'] = audio_data['mfcc_std'].apply(lambda x: np.fromstring(x.strip('[]'), sep=' '))

# Объединяем признаки
audio_data['features'] = audio_data.apply(lambda row: np.concatenate([row['mfcc_mean'], row['mfcc_std']]), axis=1)

# Переводим метки в числовой формат
bin_map = {'neut': 0, 'neg': 1, 'pos': 1}
audio_data['fin_binery'] = audio_data['fin_binery'].map(bin_map)

Обработка обучающей и тестовой выборку

In [ ]:
# Разделяем признаки и метки
X = np.stack(audio_data['features'].values)
y = audio_data['fin_binery'].values

# Делим на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=56)

# Масштабируем признаки
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование в тензоры PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Загрузка в DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
hidden_sizes = [(128, 64, 32), (64, 32, 16)]
learning_rates = [0.001, 0.0005]
batch_sizes = [8, 16]

# Все комбинации
param_grid = list(itertools.product(hidden_sizes, learning_rates, batch_sizes))

best_f1 = 0
best_model = None
best_history = None
best_params = None

Определение архитектуры и обучение модели

In [ ]:
# Функция обучения с логом истории
def train_audio_model(hidden_layers, lr, batch_size, epochs=50):
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.layers = nn.Sequential(
                nn.Linear(40, hidden_layers[0]),
                nn.ReLU(),
                nn.Linear(hidden_layers[0], hidden_layers[1]),
                nn.Dropout(0.3),
                nn.ReLU(),
                nn.Linear(hidden_layers[1], hidden_layers[2]),
                nn.ReLU(),
                nn.Linear(hidden_layers[2], 3)
            )
        def forward(self, x):
            return self.layers(x)

    model = Net()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    history = {'loss': [], 'acc': []}

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        correct = 0
        total = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)

        history['loss'].append(epoch_loss / len(train_loader))
        history['acc'].append(correct / total)

    return model, history

In [ ]:
# Поиск лучшей модели
for hidden, lr, batch in param_grid:
    model, history = train_audio_model(hidden, lr, batch)
    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor)
        pred_labels = torch.argmax(preds, dim=1).numpy()
        f1 = f1_score(y_test, pred_labels, average='macro')

    if f1 > best_f1:
        best_f1 = f1
        best_model = model
        best_history = history
        best_params = {'hidden_layers': hidden, 'lr': lr, 'batch_size': batch}


plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(best_history['loss'], label='Loss')
plt.title('Loss over epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(best_history['acc'], label='Accuracy', color='green')
plt.title('Accuracy over epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()

# Печатаем лучший F1 и параметры
print("Лучшая модель по F1-score:")
print("F1-score:", round(best_f1, 4))
print("Параметры:", best_params)

In [ ]:
# Метрики лучшей модели
best_model.eval()
with torch.no_grad():
    final_preds = best_model(X_test_tensor)
    final_labels = torch.argmax(final_preds, dim=1).numpy()

print("\nПолный отчет по метрикам:\n")
print(classification_report(y_test, final_labels, target_names=['0', '1']))